In [1]:
# WebDriver（模擬使用者在瀏覽器中的操作）

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import requests
import datetime
import os
import time
from calendar import monthrange


# Function to calculate the number of weeks a month spans, given a year and month,
def count_spanned_weeks(year, month):
    days_in_month = monthrange(year, month)[1] # Get total number of days in the month
    first_day_of_month_weekday = monthrange(year, month)[0] # Get the weekday of the first day of the month
    complete_weeks = days_in_month // 7 # Calculate the number of complete weeks
    extra_days = days_in_month % 7 # Check if there are extra days that spill over into a partial week
    spanned_weeks = complete_weeks # Start counting weeks with complete weeks
    # Check if extra days cause the month to span into an additional week
    if extra_days > 0:
        if (first_day_of_month_weekday + extra_days) > 6: # If the first day of the month + extra days spills over the week boundary, add another week
            spanned_weeks += 2  # one for the partial start week, one for the partial end week
        else:
            spanned_weeks += 1  # only the partial end week
    return spanned_weeks


def get_previous_month(month_name): # month_name: str
    month_number = datetime.datetime.strptime(month_name, '%B').month
    previous_month_number = (month_number - 2) % 12 + 1
    return previous_month_number # return: int


def select_date(driver, target_date, target_month, target_year): # 沒辦法找三月
    wait = WebDriverWait(driver, 10)
    date_picker = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "date-picker-wrapper")))

    # Check 右半邊 'month2'
    month_element = date_picker.find_elements(By.CLASS_NAME, "month-name")[1] # 右半邊
    month_name = month_element.find_elements(By.CLASS_NAME, "month-element")[0].text.lower()
    year = month_element.find_elements(By.CLASS_NAME, "month-element")[1].text
    if target_month.lower() == month_name and target_year == year:
        days = date_picker.find_elements(By.CLASS_NAME, "day")
        prev_spanned_weeks = count_spanned_weeks(int(year), get_previous_month(month_name))
        for day in days[7*prev_spanned_weeks:]:
            if day.text.zfill(2) == target_date.zfill(2):
                day.click()
                return  
        
    # Check 左半邊 'month1'
    month_element = date_picker.find_elements(By.CLASS_NAME, "month-name")[0] # 左半邊
    month_name = month_element.find_elements(By.CLASS_NAME, "month-element")[0].text.lower()
    year = month_element.find_elements(By.CLASS_NAME, "month-element")[1].text
    while target_month.lower() != month_name or target_year != year:
        next_button = date_picker.find_element(By.CLASS_NAME, "prev")
        next_button.click()
        month_element = date_picker.find_elements(By.CLASS_NAME, "month-name")[0] # 左半邊
        month_name = month_element.find_elements(By.CLASS_NAME, "month-element")[0].text.lower()
        year = month_element.find_elements(By.CLASS_NAME, "month-element")[1].text
    days = date_picker.find_elements(By.CLASS_NAME, "day")
    for day in days:
        if day.text.zfill(2) == target_date.zfill(2):
            day.click()
            break


# Open website
url = input("Input the URL: ")

# Allow driver to download data
download_directory = os.getcwd() # location where the downloaded data is saved
print(download_directory)
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    "download.default_directory": download_directory
})
driver = webdriver.Chrome(options=chrome_options) # executable_path=driver_path, 
driver.get(url)

# Select dataset
dataset_dropdown = driver.find_element(By.CSS_SELECTOR, '#dropdown')
dataset_dropdown.send_keys("Prof.Hsieh's Solar Panel")

# Select dates: 先選 end 再按 prev 選 start
select_date(driver, "2", "march", "2024")
select_date(driver, "29", "january", "2024")

select_date(driver, "21", "march", "2024")
select_date(driver, "2", "march", "2024")

# Click submit button and wait for the file to download
submit_button = driver.find_element(By.NAME, 'export')
submit_button.click()
time.sleep(5)

# Quit the driver
driver.quit()


c:\Users\huyushin\MY_FILES\PYTHON_FILES\NTU_Meter_Crawler\deprecated_scripts


In [ ]:
## Get page source and save to file
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
# page_content = driver.page_source
# with open('page_content.html', 'w', encoding='utf-8') as f:
#     f.write(page_content)
# print("Page content has been saved to 'page_content.html'")


## Select the latest downloaded file and rename it to csv format
# files = os.listdir(download_directory)
# for file in files:
#     if file.endswith('.tmp'):
#         tmp_file_path = os.path.join(download_directory, file)
#         csv_file_path = os.path.join(download_directory, file.replace('.tmp', '.csv'))
#         os.rename(tmp_file_path, csv_file_path)
#         print("Renamed file to:", csv_file_path)
#         break
